<a href="https://colab.research.google.com/github/whispermy/novatus_academia/blob/main/pbl_lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### 필요 패키지 불러오기

import numpy as np
import tensorflow.keras as keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [2]:
# 데이터셋 받아와서 charcter 의 모음인 string 으로 저장하기

shakespeare_url = "https://homl.info/shakespeare" 
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

In [3]:
# 각 글자를 정수로 코딩하기

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [4]:
tokenizer.texts_to_sequences(["Apple"])

[[5, 23, 23, 12, 2]]

In [5]:
tokenizer.sequences_to_texts([[5, 23, 23, 12, 2]])

['a p p l e']

In [6]:
max_id=len(tokenizer.word_index)
max_id

39

In [7]:
# 전체 글자의 개수 

dataset_size=tokenizer.document_count

In [8]:
dataset_size

1115394

In [9]:
tokenizer.word_index

{'\n': 11,
 ' ': 1,
 '!': 31,
 '$': 39,
 '&': 38,
 "'": 28,
 ',': 18,
 '-': 32,
 '.': 27,
 '3': 37,
 ':': 24,
 ';': 29,
 '?': 30,
 'a': 5,
 'b': 22,
 'c': 19,
 'd': 13,
 'e': 2,
 'f': 20,
 'g': 21,
 'h': 7,
 'i': 6,
 'j': 33,
 'k': 25,
 'l': 12,
 'm': 15,
 'n': 10,
 'o': 4,
 'p': 23,
 'q': 34,
 'r': 9,
 's': 8,
 't': 3,
 'u': 14,
 'v': 26,
 'w': 17,
 'x': 35,
 'y': 16,
 'z': 36}

In [10]:
#학습용 데이터셋 분리하기

train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [11]:
# window() 함수 이용하여 하나의 긴 글을 여러 구간으로 분리

n_steps = 30
window_length = n_steps + 1 
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [12]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, -1]))

In [13]:
# 각 정수 값을 one-hot 벡터로 바꾸기

dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

dataset = dataset.prefetch(1) # 데이터 prefetching

In [14]:
dataset

<PrefetchDataset shapes: ((None, None, 39), (None,)), types: (tf.float32, tf.int64)>

In [15]:
model = keras.models.Sequential()
model.add(keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],dropout=0.2, recurrent_dropout=0))
model.add(keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0))
# model.add(keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax")))
model.add(keras.layers.GRU(max_id, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)

Epoch 1/10
   1224/Unknown - 158s 124ms/step - loss: 2.5899

KeyboardInterrupt: ignored